In [1]:
from AuT.lib.dataset import FewShotSpeechCommand

dataset = FewShotSpeechCommand(
    root_path='/home/andyshao/data/speech_commands',
    mode='test', include_rate=False, data_tfs=None
)
len(dataset)

6835

In [2]:
dataset = FewShotSpeechCommand(
    root_path='/home/andyshao/data/speech_commands',
    mode='test-fs', include_rate=False, data_tfs=None
)
len(dataset)

300

In [3]:
dataset = FewShotSpeechCommand(
    root_path='/home/andyshao/data/speech_commands',
    mode='test-res', include_rate=False, data_tfs=None
)
len(dataset)

6535